# 1) Exploring dataset

### 1.1) Import libraries

In [1]:
import numpy as np
import pandas as pd
import datetime

### 1.2) Read dataset

Dataset: https://www.kaggle.com/barun2104/government-measures-to-combat-covid19

In [2]:
df = pd.read_csv('./input/covid.csv', encoding='latin-1')

### 1.3) Explore dataset

In [3]:
display(df.head(5))

,ID,COUNTRY,ISO,ADMIN_LEVEL_NAME,PCODE,REGION,LOG_TYPE,CATEGORY,MEASURE,TARGETED_POP_GROUP,...,SOURCE,SOURCE_TYPE,LINK,ENTRY_DATE,Alternative source,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22
0,1.0,Afghanistan,AFG,NaN,NaN,Asia,Introduction / extension of measures,Public health measures,Health screenings in airports and border cross...,No,...,Ministry of Health,Government,https://moph.gov.af/en/moph-held-emergency-mee...,14-03-2020,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,Afghanistan,AFG,Kabul,NaN,Asia,Introduction / extension of measures,Public health measures,Introduction of isolation and quarantine policies,No,...,Ministry of Health,Government,https://moph.gov.af/en/moph-held-emergency-mee...,14-03-2020,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,Afghanistan,AFG,NaN,NaN,Asia,Introduction / extension of measures,Public health measures,Awareness campaigns,No,...,Ministry of Health,Government,https://moph.gov.af/en/moph-held-emergency-mee...,14-03-2020,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,Afghanistan,AFG,NaN,NaN,Asia,Introduction / extension of measures,Governance and socio-economic measures,Emergency administrative structures activated ...,No,...,Ministry of Health,Government,https://moph.gov.af/en/moph-held-emergency-mee...,14-03-2020,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,Afghanistan,AFG,NaN,NaN,Asia,Introduction / extension of measures,Social distancing,Limit public gatherings,No,...,AA,Media,https://www.aa.com.tr/en/asia-pacific/coronavi...,14-03-2020,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.shape

(5072, 23)

In [5]:
df.isnull().sum()

ID                     619
COUNTRY                619
ISO                    619
ADMIN_LEVEL_NAME      4820
PCODE                 5072
REGION                 619
LOG_TYPE               619
CATEGORY               620
MEASURE                619
TARGETED_POP_GROUP     632
COMMENTS               763
NON_COMPLIANCE        2067
DATE_IMPLEMENTED       798
SOURCE                 636
SOURCE_TYPE            637
LINK                   648
ENTRY_DATE             619
Alternative source    4721
Unnamed: 18           5072
Unnamed: 19           5072
Unnamed: 20           5072
Unnamed: 21           5072
Unnamed: 22           5072
dtype: int64

# 2) Clean dataset

### 2.1) Select columns of interest

In [6]:
columns=['COUNTRY', 'ISO', 'REGION','CATEGORY', 'MEASURE', 'COMMENTS', 'SOURCE', 'ENTRY_DATE']

In [7]:
df=df[columns]

In [8]:
df.head(5)

,COUNTRY,ISO,REGION,CATEGORY,MEASURE,COMMENTS,SOURCE,ENTRY_DATE
0,Afghanistan,AFG,Asia,Public health measures,Health screenings in airports and border cross...,NaN,Ministry of Health,14-03-2020
1,Afghanistan,AFG,Asia,Public health measures,Introduction of isolation and quarantine policies,NaN,Ministry of Health,14-03-2020
2,Afghanistan,AFG,Asia,Public health measures,Awareness campaigns,NaN,Ministry of Health,14-03-2020
3,Afghanistan,AFG,Asia,Governance and socio-economic measures,Emergency administrative structures activated ...,NaN,Ministry of Health,14-03-2020
4,Afghanistan,AFG,Asia,Social distancing,Limit public gatherings,Nevruz festival cancelled,AA,14-03-2020


### 2.2) Drop duplicates

In [9]:
before = len(df)
df = df.drop_duplicates()
after = len(df)
print(f'Number of duplicate records dropped: {before - after}')

Number of duplicate records dropped: 625


In [10]:
df.shape

(4447, 8)

### 2.3) Drop rows with null values

In [11]:
before = df.shape[0]
df = df[df.isnull().sum(axis=1) < 5]
after = df.shape[0]
print(f'Number of rows dropped: {before - after}')

Number of rows dropped: 1


### 2.4) Filling Null Values

In [12]:
df=df.fillna('Unknown')

In [13]:
df=df.reset_index(drop=True)

### 2.5) Correcting values in columns

#### 2.5.1) Correcting ENTRY_DATE column

The data type in the ENTRY_DATE column is a string. It will be transformed to datetime. It will be easier to work with this column later this way. 

In [14]:
for i,e in df['ENTRY_DATE'].items():
    fecha=e.split('-')
    fecha=[fecha[2],fecha[1],fecha[0]]
    fecha=[int(e) for e in fecha]
    df['ENTRY_DATE'][i]=datetime.date(*fecha)

df['ENTRY_DATE']

0       2020-03-14
1       2020-03-14
2       2020-03-14
3       2020-03-14
4       2020-03-14
           ...    
4441    2020-04-09
4442    2020-04-09
4443    2020-04-09
4444    2020-04-09
4445    2020-04-09
Name: ENTRY_DATE, Length: 4446, dtype: object

#### 2.5.2) Correcting ISO column

The [API](https://about-corona.net/documentation) used in this project provides information about the COVID evolution throughout time for a specific country. It requests a string paremeter with the country code in [ISO 3166-1 alpha-2 format](https://en.wikipedia.org/wiki/ISO_3166-1_alpha-2). For this reason, the ISO column will be modified to meet these requirements. 

In [38]:
import pycountry
input_countries=list(df['COUNTRY'].unique())
countries = {}
for country in pycountry.countries:
    countries[country.name] = country.alpha_2
#countries
df["ISO_ALP_2"] = df["COUNTRY"]
df=df.replace({"ISO_ALP_2": countries})

df.head()

,COUNTRY,ISO,REGION,CATEGORY,MEASURE,COMMENTS,SOURCE,ENTRY_DATE,ISO_2
0,Afghanistan,AF,Asia,Public health measures,health screenings in airports and border cross...,Unknown,ministry of health,2020-03-14,AF
1,Afghanistan,AF,Asia,Public health measures,introduction of isolation and quarantine policies,Unknown,ministry of health,2020-03-14,AF
2,Afghanistan,AF,Asia,Public health measures,awareness campaigns,Unknown,ministry of health,2020-03-14,AF
3,Afghanistan,AF,Asia,Governance and socio-economic measures,emergency administrative structures activated ...,Unknown,ministry of health,2020-03-14,AF
4,Afghanistan,AF,Asia,Social distancing,limit public gatherings,Nevruz festival cancelled,aa,2020-03-14,AF


In [15]:
import pycountry

countries = {}
for country in pycountry.countries:
    countries[country.name] = country.alpha_2

alpha2f = {
           'Dec' : 'Dec-Feb', 'Jan' : 'Dec-Feb', 'Feb' : 'Dec-Feb',
           'Mar' : 'Mar-May', 'Apr' : 'Mar-May', 'May' : 'Mar-May',
           'Jun' : 'Jun-Aug', 'Jul' : 'Jun-Aug', 'Aug' : 'Jun-Aug',
           'Sep' : 'Sep-Nov', 'Oct' : 'Sep-Nov', 'Nov' : 'Sep-Nov', 
           }

df=df.replace("ISO": alpha2f)


for i,e in df['ISO'].items():
    df['ISO'][i]=e[:2]

#### 2.5.3) Correcting MEASURE column

In [16]:
df['MEASURE'].value_counts()

Economic measures                                               489
Limit public gatherings                                         427
Strengthening the public health system                          391
Introduction of isolation and quarantine policies               391
International flights suspension                                244
Visa restrictions                                               238
Border closureÿ                                                 236
Public services closureÿ                                        224
General recommendations                                         200
Schools closureÿ                                                190
Awareness campaigns                                             186
Health screenings in airports and border crossings              177
Domestic travel restrictions                                    155
Emergency administrative structures activated or established    139
Curfews                                         

In [17]:
for i,e in df['MEASURE'].items():
    df['MEASURE'][i]=e.lower()
    if 'ÿ' in e:
        df['MEASURE'][i]=e.replace('ÿ','')

#### 2.5.4) Correcting SOURCE column

In [18]:
for i,e in df['SOURCE'].items():
    df['SOURCE'][i]=e.lower()
    if '.' in e:
        df['SOURCE'][i]=e.replace('.','')

## 3) Save Clean dataset

The clean dataset will be saved in the output folder for later analysis.

In [28]:
df[df['ISO']=='AR']

,COUNTRY,ISO,REGION,CATEGORY,MEASURE,COMMENTS,SOURCE,ENTRY_DATE
23,Argentina,AR,Americas,Movement restrictions,international flights suspension,Flights from countries with COVID transmissions.,ministry of foreign affairs - france,2020-03-14
24,Argentina,AR,Americas,Movement restrictions,visa restrictions,For long term,ministry of foreign affairs - francce,2020-03-14
25,Argentina,AR,Americas,Public health measures,introduction of isolation and quarantine policies,"From Europe, UK, USA, South Korea, Japan, Chin...",ministry of foreign affairs - francce,2020-03-14
61,Armenia,AR,Asia,Public health measures,health screenings in airports and border cross...,New arrivals also fill out information forms,government of the republic of armenia,2020-03-14
63,Armenia,AR,Asia,Movement restrictions,Border closure,iran for 2 weeks,us embassy,2020-03-14
...,...,...,...,...,...,...,...,...
3708,Armenia,AR,Asia,Public health measures,introduction of isolation and quarantine policies,Self-isolation is mandatory for all persons an...,international sos,2020-04-03
3709,Armenia,AR,Asia,Movement restrictions,visa restrictions,"citizens of Australia, Canada, China, EU membe...",international sos,2020-04-03
4415,United Arab Emirates,AR,Middle East,Governance and socio-economic measures,emergency administrative structures activated ...,Federal Law No. 14 of 2014 on Communicable Dis...,global states of emergency,2020-04-09
4442,Armenia,AR,Asia,Lockdown,partial lockdown,only be allowed to leave their homes to buy fo...,global states of emergency,2020-04-09


In [20]:
df.to_csv("output/clean.csv")